# Message Queue: RabbitMQ Pika Publisher
## Illustrates distributed computing using message queues.

We will use multiple implementations of message queues including <b>RabbitMQ/pika</b> and <b>MQTT</b>. This notebook is an illustration of publishing or sending a stream using RabbitMQ.
<br>
<br>
The publisher and subscriber communicate through a specified <i>routing key</i>, <i>exchange</i>, and <i>host</i>. In this example, the data consists of a list of lists, and each sublist of data is published separately after which the sender pauses.
<br>
<br>
<b>Note</b>: Run the notebook <i>ExamplePikaSubscriber</i> first. Then run this notebook: <i>ExamplePikaPublisher</i>. You should see the data that you published using this notebook printed in the notebook <i>ExamplePikaSubscriber</i>.


# Publishing a Stream: The Central Idea
## PikaPublisher(routing_key, exchange, host).publish(stream)
The central idea is given by the line above. Next we give an example using threads; you can use the example if you want to publish a stream in an application with multiple processes.

# An Example

In [1]:
import pika
import json
import time
import threading

import sys
sys.path.append("../")
from IoTPy.core.agent import Agent
from IoTPy.core.stream import Stream, StreamArray, _no_value, _multivalue, run
from IoTPy.agent_types.op import map_element
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.helper_functions.print_stream import print_stream
# multicore imports
from IoTPy.concurrency.multicore import get_processes, get_processes_and_procs
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import get_proc_that_inputs_source
from IoTPy.concurrency.multicore import extend_stream
# Pika imports
from IoTPy.concurrency.PikaPublisher import PikaPublisher
from IoTPy.concurrency.PikaSubscriber import PikaSubscriber

In [2]:
def test_pika_publisher(routing_key, exchange, host, data):
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.

    # pika_publisher_agent is the agent for the processor
    # called 'pika_publisher_process'.
    def pika_publisher_agent(in_streams, out_streams):
        # publish in_streams[0] for the specified routing key, exchange, host.
        PikaPublisher(
            routing_key, exchange, host).publish(in_streams[0])

    # Step 0.1: Define source thread targets (if any).
    def thread_target_source(procs):
        for sublist in data:
            extend_stream(procs, data=sublist, stream_name='source')
            # Sleep to simulate an external data source.
            time.sleep(0.001)
        # Put '_finished' on the stream because the stream will not
        # be extended. This informs subscriber that stream is finished.
        extend_stream(procs, data=['_finished'], stream_name='source')
        # Terminate stream because this stream will not be extended.
        terminate_stream(procs, stream_name='source')

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs, additional arguments.
    multicore_specification = [
        # Streams
        [('source', 'x')],
        # Processes
        [{'name': 'pika_publisher_process', 'agent': pika_publisher_agent, 
          'inputs':['source'], 'sources': ['source']}]]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    thread_0 = threading.Thread(target=thread_target_source, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # thread_0 runs in the process called 'p1'
    procs['pika_publisher_process'].threads = [thread_0]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

In [3]:
# This is some arbitrary data merely for testing.
# Each sublist of data is published separately.
data = [[0, 1], ['Hello', 'World'], ['THE', 'END', 'IS', 'NIGH!', '_finished']]
    
test_pika_publisher(
    routing_key='temperature', exchange='publications', host='localhost', 
    data=data)